<a href="https://colab.research.google.com/github/Reynxzz/capstone-ipb/blob/main/Capstone_IPB_Making_Dataset_%26_Recommender_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Making Dataset

STEP bikin dataset:
1. Bikin random point di google earth sekitaran IPB
2. Export latitude dan longitude dari point2 itu
3. Generate random route (cyclic) dengan start Agrimart dan Rektorat
4. Sesuaikan distancenya (max 3 km dan max 5 km)
5. Ambil data elevation per point subset tersebut pake Open Topodata API
6. Sesuaikan kesulitannya pake data elevasi (Easy, Medium, Hard) --> lagi disini
7. Sesuaikan dengan pemandangan

Expected dataset:

| Route | (lat,lon) | Distance | Duration | Elevation Max | Elevation Min | Elevation Difference | Difficulty | Scenery | Preview(image) |

###Import CSV datasets from Google Earth

In [1]:
import csv

# Open the CSV file for reading
with open('/content/drive/My Drive/Capstone IPB/Tempat_di_IPB.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)

    # Create a list of tuples containing latitude and longitude
    coordinates = [(float(row['Y']), float(row['X'])) for row in reader]

Create dataframe

In [2]:
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/Capstone IPB/Tempat_di_IPB.csv')

In [3]:
# remove uneccessary column
df = df.drop(df.columns[-1], axis=1)

# change column name
df = df[['Name', 'Y', 'X']]
df = df.rename(columns={'Name': 'name', 'X': 'longitude', 'Y': 'latitude'})
df

,name,latitude,longitude
0,Starting Point 1 - Agrimart,-6.559847,106.731220
1,Depan Koin IPB,-6.560860,106.727949
2,Depan Rektorat,-6.560678,106.725458
3,Pintu Masuk ke Kiri,-6.557232,106.719038
4,Warteg Sebelah Alhur,-6.556168,106.724086
5,Depan Alhur,-6.556064,106.725050
6,Perumdos 1,-6.552980,106.721197
7,Belakang 1,-6.552237,106.717024
8,Sawah 1,-6.551354,106.714950
9,Belakang Banget,-6.547448,106.715541


Visualize Dataframe

In [5]:
import folium
# create a map centered on the first location
m_raw = folium.Map(location=[df['latitude'][0], df['longitude'][0]], zoom_start=10)

# add markers for each location in the DataFrame
for index, row in df.iterrows():
    folium.Marker(location=[row['latitude'], row['longitude']], popup=row['name']).add_to(m_raw)

# display the map
m_raw

###Generate Routes

Bikin rute random

In [6]:
import random
import json
import requests

def routes_maker(total_point, max_distance, threshold=300):
    subset = []
    # define start and end coordinates
    start = coordinates[0]
    end = coordinates[0]

    # randomly select a subset of coordinates
    subset = random.sample(coordinates, k=total_point)  # k is the number of coordinates in the subset

    # add the start and end coordinates to the beginning and end of the subset
    subset.insert(0, start)
    subset.append(end)

    api_url = 'https://api.openrouteservice.org/v2/directions/foot-walking'
    api_key = '5b3ce3597851110001cf6248d4725ac662f8461eb589679758fb20ea'

    locations = [list(r) for r in subset]
    total_distance = 0
    total_duration = 0

    for i in range(len(locations)-1):
        # Define the start and end coordinates for the API call
        start_coords = f"{locations[i][1]},{locations[i][0]}"
        end_coords = f"{locations[i+1][1]},{locations[i+1][0]}"
        
        # Define the API call parameters, including the start and end coordinates, the API key, and the travel mode (walking)
        params = {
            'api_key': api_key,
            'start': start_coords,
            'end': end_coords,
            'profile': 'foot-walking',
        }
        
        # Make the API call and parse the response as JSON
        response = requests.get(api_url, params=params)
        data = json.loads(response.text)
        # print(data)
        # Extract the walking distance and duration from the response
        distance = data['features'][0]['properties']['segments'][0]['distance']
        duration = data['features'][0]['properties']['segments'][0]['duration']
        # elevation = data['features'][0]['properties']['segments'][0]['steps'][0]['elevation']

        print(f"Distance between locations {i+1} and {i+2}: {distance:.2f} meters")

        # total distance and duration
        total_distance += distance
        total_duration += duration
        
        # visualize the route
        m = folium.Map(location=start, zoom_start=14)
        folium.PolyLine(locations=[list(reversed(coord)) 
                              for coord in 
                              data['features'][0]['geometry']['coordinates']]).add_to(m)

    # print total distance and duration
    print(f"Total distance: {total_distance:.2f} meters")
    print(f"Total duration: {total_duration:.2f} seconds")
    
    # add marker to map
    folium.Marker(location=start, popup='Start here', icon=folium.Icon(color='green', prefix='fa', icon='running')).add_to(m)
    folium.Marker(location=locations[len(locations) // 2], popup='Finish here (turn around)', icon=folium.Icon(color='blue',prefix='fa', icon='undo')).add_to(m)

    # if requirements not satisfied, run the function again!
    if total_distance < max_distance + threshold:
      return total_distance, m, subset 
    else:      
      return routes_maker(total_point, max_distance)

In [7]:
total_distances, m, subset = routes_maker(2, 5000)

Distance between locations 1 and 2: 2329.30 meters
Distance between locations 2 and 3: 949.90 meters
Distance between locations 3 and 4: 3071.90 meters
Total distance: 6351.10 meters
Total duration: 4572.70 seconds
Distance between locations 1 and 2: 1079.10 meters
Distance between locations 2 and 3: 668.90 meters
Distance between locations 3 and 4: 1639.30 meters
Total distance: 3387.30 meters
Total duration: 2438.90 seconds


In [76]:
routes = [total_distances] + subset 
routes

[3387.3,
 (-6.55984667674852, 106.731220037341),
 (-6.5561681108102, 106.724086099391),
 (-6.55723238992818, 106.719037577124),
 (-6.55984667674852, 106.731220037341)]

Elevation Max, Min, Difference

In [17]:
def get_elevations(coordinates):
    """Lihat dokumentasi disini bro https://www.opentopodata.org/"""

    # Build the API URL for the OpenTopo API
    url = "https://api.opentopodata.org/v1/aster30m"

    # Make the API call for each coordinate and extract the elevation data
    elevations = []
    for coord in coordinates:
        lat, lon = coord
        query = f"{url}?locations={lat},{lon}"
        response = requests.get(query)
        if response.status_code == 200:
            data = response.json()
            elevation = data["results"][0]["elevation"]
            elevations.append(elevation)
        else:
            elevations.append(None)

    # Return the list of elevations
    return elevations

In [18]:
subset_elevations = get_elevations(subset)
subset_elevations

[184.0, 182.0, 176.0, 184.0]

In [19]:
max_elevation = max(subset_elevations)
min_elevation = min(subset_elevations)
elevation_difficulty = max_elevation - min_elevation

max_elevation, min_elevation, elevation_difficulty

(184.0, 176.0, 8.0)

Initial Dataframe

In [95]:
import numpy as np
# Create a dictionary with the data and column names
routes_dict = {
              'id': np.random.randint(100, 1000),
              'distance (m)': routes[0],
             'route_coordinate (lat,lon)': [routes[1:5]],
             'max_elevation (m)': max_elevation,
             'min_elevation (m)': min_elevation,
             'difficulty (m)': elevation_difficulty}

In [96]:
routes_dict

{'id': 609,
 'distance (m)': 3387.3,
 'route_coordinate (lat,lon)': [[(-6.55984667674852, 106.731220037341),
   (-6.5561681108102, 106.724086099391),
   (-6.55723238992818, 106.719037577124),
   (-6.55984667674852, 106.731220037341)]],
 'max_elevation (m)': 184.0,
 'min_elevation (m)': 176.0,
 'difficulty (m)': 8.0}

In [97]:
df_routes = pd.DataFrame(routes_dict)

In [98]:
df_routes

,id,distance (m),"route_coordinate (lat,lon)",max_elevation (m),min_elevation (m),difficulty (m)
0,609,3387.3,"[(-6.55984667674852, 106.731220037341), (-6.55...",184.0,176.0,8.0


Visualisasi rute

In [99]:
m

Experiment

In [ ]:
!pip install osmnx

In [ ]:
import osmnx as ox

# Define the location of interest using an address or place name
location_name = 'San Francisco, California'

# Define the distance (in meters) around the location to retrieve OSM data for
distance = 500

# Download and construct a network from OSM data within the specified distance from the location
G = ox.graph_from_address(location_name, distance, network_type='all')

# Calculate the area of the network in square meters
area = ox.set_graph_crs(G, 'EPSG:4326').unary_union.area

# Calculate the amount of tree cover within the network
trees = ox.geometries_from_address(location_name, tags={'natural': 'tree'})
tree_cover = trees.geometry.unary_union.area / area

print(f"Total area: {area:.2f} square meters")
print(f"Tree cover: {tree_cover:.2%}")

AttributeError: ignored